In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression
import sys
sys.path.append("./models")
import importlib
import sparse_fwfm
importlib.reload(sparse_fwfm)
FwFM = sparse_fwfm.FwFM

In [2]:
path = './data/'
df = pd.read_csv(path + "CrossDomain_2years.csv", index_col=0)
df.head()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,12,13,14,15,16,17,18,27,28,29,30,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,index,SEX,AGE,AREA,Salary,domein,job[0],job[1],job[2],job[3],...,importance[15],importance[16],importance[17],importance[18],Company,most_importance,question,sentence,factor,SCORE
0,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_1,申込み窓口の豊富さ（取扱い店舗、ネット申込み、保険営業員 等）,加入手続き,6
1,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_2,加入手続きの容易さ（記入事項の分かりやすさ、提出書類の内容、情報提供、等を含む）,加入手続き,6
2,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_3,加入手続き完了まで（契約完了まで）のスピード,加入手続き,6
3,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_4,商品内容の分かりやすさ,商品内容,6
4,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_5,商品内容の充実度（特約等を含む）,商品内容,6


In [13]:
df_count = df.groupby(["domein", "Company"])["index"].count().reset_index().rename(columns = {"index":"count"})
df = pd.merge(df, df_count, on=["domein", "Company"])
df = df[df["count"] > 100]#.drop("count",axis=1)

In [14]:
df_X = df[["SEX", "AGE", "Salary", "domein", "job[0]", "job[1]", "importance[0]", "importance[1]", "Company", "question", "sentence", "factor"]].fillna("Nan").copy()
df_y = df["SCORE"].copy()

In [ ]:
group_kfold = GroupKFold(n_splits=3)
userid = df["index"].values
for train_index, test_index in group_kfold.split(df_X, df_y, userid):
    df_X_train, df_y_train = df_X.iloc[train_index], df_y.iloc[train_index]
    df_X_test, df_y_test = df_X.iloc[test_index], df_y.iloc[test_index]
    
    sample_weight = np.clip(1 / df["count"].iloc[train_index].values, 0, 100)
    sample_weight = (sample_weight - sample_weight.min()) / (sample_weight.max() - sample_weight.min())
    
    fwfm = FwFM(df_X_train, df_y_train, sample_weight = sample_weight, n_epoch=10, dim=8)
    y_hat = fwfm(df_X_test)

    #fwfm_score = mean_squared_error(df_y_test.values, y_hat * std + mean) ** 0.5
    fwfm_score = mean_squared_error(df_y_test.values, y_hat) ** 0.5

    reg = LinearRegression()
    reg.fit(fwfm.X, fwfm.y)
    X_test = fwfm.convert_sparse(df_X_test)
    y_hat = reg.predict(X_test)
    #linear_score = mean_squared_error(df_y_test.values, y_hat * std + mean) ** 0.5
    linear_score = mean_squared_error(df_y_test.values, y_hat) ** 0.5

    print(f"fwfm = {fwfm_score}, linear = {linear_score}") 
    break   

 28%|██▊       | 187/679 [00:10<00:28, 17.05it/s]

In [11]:
y_hat = fwfm(df_X_test)
y_hat_linear = reg.predict(X_test)
df_X_test_eval = df_X_test.copy()
df_X_test_eval["fwfm_score"] = y_hat
df_X_test_eval["linear_score"] = y_hat_linear
df_X_test_eval["SCORE"] = df_y_test
df_eval_gby = df_X_test_eval.groupby(["Company","factor","domein"])[["SCORE","fwfm_score", "linear_score"]].mean().reset_index()
df_eval_gby["fwfm_squared_error"] = (df_eval_gby["fwfm_score"] - df_eval_gby["SCORE"]) ** 2
df_eval_gby["linear_squared_error"] = (df_eval_gby["linear_score"] - df_eval_gby["SCORE"]) ** 2
df_eval_gby["fwfm_root_mean_squared_error"] = df_eval_gby["fwfm_squared_error"] ** 0.5
df_eval_gby["linear_root_mean_squared_error"] = df_eval_gby["linear_squared_error"] ** 0.5

df_eval = df_eval_gby.groupby("domein").mean(["fwfm_squared_error", "linear_squared_error"]).reset_index()[["domein", "fwfm_root_mean_squared_error", "linear_root_mean_squared_error"]]
display(df_eval)
df_eval.mean(axis=0)

100%|██████████| 84/84 [00:00<00:00, 124.21it/s]


,domein,fwfm_root_mean_squared_error,linear_root_mean_squared_error
0,1-1-18：医療保険,0.390372,0.348105
1,1-1-19：医療保険,0.357918,0.366350
2,1-2-17：海外旅行保険,0.351478,0.372760
3,1-2-18：海外旅行保険,0.352858,0.386199
4,1-4-18：がん保険,0.459411,0.451401
5,1-4-19：がん保険,0.793081,0.794671
6,1-7-18：生命保険,0.359783,0.342405
7,1-7-19：生命保険,0.324668,0.355950
8,1-9-17：ペット保険,0.479858,0.558250
9,1-9-18：ペット保険,0.512149,0.521644


fwfm_root_mean_squared_error      0.438158
linear_root_mean_squared_error    0.449773
dtype: float64